<a href="https://colab.research.google.com/github/alinecorreia22/Otimiza-o-de-Entrevistas-com-IA/blob/main/parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Execute e selecione o arquivo .json correto do seu computador
uploaded = files.upload()


Saving applicants.json to applicants.json
Saving vagas.json to vagas.json
Saving prospects.json to prospects.json


In [ ]:
import pandas as pd
import json

# --- ETAPA 1: Transformar 'aplicacoes.json' ---
with open('applicants.json', 'r', encoding='utf-8') as f:
    aplicacoes_raw = json.load(f)

aplicacoes_df = pd.json_normalize(aplicacoes_raw).T.reset_index()
aplicacoes_df[['id_aplicacao', 'campo']] = aplicacoes_df['index'].str.extract(r"^(\d+)\.(.*)")
aplicacoes_df = aplicacoes_df.pivot(index='id_aplicacao', columns='campo', values=0).reset_index()

# --- ETAPA 2: Transformar 'vagas.json' ---
with open('vagas.json', 'r', encoding='utf-8') as f:
    vagas_raw = json.load(f)

vagas_df = pd.json_normalize(vagas_raw).T.reset_index()
vagas_df[['id_vaga', 'campo']] = vagas_df['index'].str.extract(r"^(\d+)\.(.*)")
vagas_df = vagas_df.pivot(index='id_vaga', columns='campo', values=0).reset_index()

# --- ETAPA 3: Tentar detectar automaticamente a coluna de vínculo ---
possiveis_chaves_aplicacao = [col for col in aplicacoes_df.columns if 'vaga' in col.lower() or 'codigo' in col.lower()]
possiveis_chaves_vaga = [col for col in vagas_df.columns if 'id' in col.lower() or 'codigo' in col.lower() or 'vaga' in col.lower()]

print("🔍 Possíveis chaves em 'aplicacoes':", possiveis_chaves_aplicacao)
print("🔍 Possíveis chaves em 'vagas':", possiveis_chaves_vaga)

# Tentar encontrar uma chave em comum (valores coincidentes)
chave_encontrada = None
for col_ap in possiveis_chaves_aplicacao:
    for col_vg in possiveis_chaves_vaga:
        if aplicacoes_df[col_ap].isin(vagas_df[col_vg]).any():
            chave_encontrada = (col_ap, col_vg)
            break
    if chave_encontrada:
        break

if chave_encontrada:
    col_ap, col_vg = chave_encontrada
    print(f"\n✅ Chave encontrada: aplicacoes['{col_ap}'] ↔ vagas['{col_vg}']")
    df_final = aplicacoes_df.merge(vagas_df, how='left', left_on=col_ap, right_on=col_vg, suffixes=('', '_vaga'))
    print(f"\n🟢 Merge realizado com sucesso. Shape final: {df_final.shape}")
else:
    print("\n❌ Nenhuma chave comum encontrada para merge. Verifique os dados manualmente.")


🔍 Possíveis chaves em 'aplicacoes': ['infos_basicas.codigo_profissional']
🔍 Possíveis chaves em 'vagas': ['id_vaga', 'informacoes_basicas.objetivo_vaga', 'informacoes_basicas.origem_vaga', 'informacoes_basicas.prioridade_vaga', 'informacoes_basicas.titulo_vaga', 'informacoes_basicas.vaga_sap', 'perfil_vaga.areas_atuacao', 'perfil_vaga.bairro', 'perfil_vaga.cidade', 'perfil_vaga.competencia_tecnicas_e_comportamentais', 'perfil_vaga.demais_observacoes', 'perfil_vaga.equipamentos_necessarios', 'perfil_vaga.estado', 'perfil_vaga.faixa_etaria', 'perfil_vaga.habilidades_comportamentais_necessarias', 'perfil_vaga.horario_trabalho', 'perfil_vaga.local_trabalho', 'perfil_vaga.nivel profissional', 'perfil_vaga.nivel_academico', 'perfil_vaga.nivel_espanhol', 'perfil_vaga.nivel_ingles', 'perfil_vaga.outro_idioma', 'perfil_vaga.pais', 'perfil_vaga.principais_atividades', 'perfil_vaga.regiao', 'perfil_vaga.vaga_especifica_para_pcd', 'perfil_vaga.viagens_requeridas']

✅ Chave encontrada: aplicacoes['

In [ ]:
print(df_final.columns.tolist())

['id_aplicacao', 'cargo_atual.cargo_atual', 'cargo_atual.cliente', 'cargo_atual.data_admissao', 'cargo_atual.data_ultima_promocao', 'cargo_atual.email_corporativo', 'cargo_atual.email_superior_imediato', 'cargo_atual.id_ibrati', 'cargo_atual.nome_superior_imediato', 'cargo_atual.projeto_atual', 'cargo_atual.unidade', 'cv_en', 'cv_pt', 'formacao_e_idiomas.ano_conclusao', 'formacao_e_idiomas.cursos', 'formacao_e_idiomas.instituicao_ensino_superior', 'formacao_e_idiomas.nivel_academico', 'formacao_e_idiomas.nivel_espanhol', 'formacao_e_idiomas.nivel_ingles', 'formacao_e_idiomas.outro_curso', 'formacao_e_idiomas.outro_idioma', 'informacoes_pessoais.cpf', 'informacoes_pessoais.data_aceite', 'informacoes_pessoais.data_nascimento', 'informacoes_pessoais.download_cv', 'informacoes_pessoais.email', 'informacoes_pessoais.email_secundario', 'informacoes_pessoais.endereco', 'informacoes_pessoais.estado_civil', 'informacoes_pessoais.facebook', 'informacoes_pessoais.fonte_indicacao', 'informacoes_pe

In [ ]:
[col for col in df_final.columns if 'status' in col.lower()]

[]

In [ ]:
[col for col in df_final.columns if any(palavra in col.lower() for palavra in ['avaliacao', 'resultado', 'entrevista', 'motivo', 'decisao', 'andamento', 'final'])]

['informacoes_basicas.data_final']

In [ ]:
colunas = [
    'formacao_e_idiomas.nivel_academico',
    'formacao_e_idiomas.nivel_ingles',
    'informacoes_profissionais.nivel_profissional',
    'informacoes_profissionais.remuneracao',
    'perfil_vaga.competencia_tecnicas_e_comportamentais',
    'informacoes_basicas.tipo_contratacao',
    'informacoes_basicas.cliente',
    'informacoes_basicas.data_final'  # usada como base para target
]

df_modelo = df_final[colunas].copy()

# Criar variável-alvo com base na presença de data_final
df_modelo['target'] = df_modelo['informacoes_basicas.data_final'].notna().astype(int)

# Remover a coluna usada como base para o target
df_modelo = df_modelo.drop(columns=['informacoes_basicas.data_final'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

X = df_modelo.drop('target', axis=1)
y = df_modelo['target']

le_dict = {}
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        le_dict[col] = le

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Salvar modelo e recursos
joblib.dump(model, 'modelo_entrevista.pkl')
joblib.dump(le_dict, 'label_encoders.pkl')
joblib.dump(list(X.columns), 'features_utilizadas.pkl')


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7582
           1       0.81      0.79      0.80       915

    accuracy                           0.96      8497
   macro avg       0.89      0.89      0.89      8497
weighted avg       0.96      0.96      0.96      8497



['features_utilizadas.pkl']